# 📝 ScratchpadTools - Agent Memory System

```mermaid
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#F1C40F', 'primaryTextColor':'#000', 'primaryBorderColor':'#F39C12', 'lineColor':'#E67E22', 'secondaryColor':'#3498DB', 'tertiaryColor':'#27AE60', 'fontSize':'16px'}}}%%
graph TB
    A[👤 User: Remember this] --> B[🤖 Agent]
    B --> C{✍️ WriteToScratchPad}
    C --> D[💾 Memory Storage]
    
    E[👤 User: What did I say?] --> F[🤖 Agent]
    F --> G{📖 ReadFromScratchPad}
    G --> D
    D --> H[📤 Retrieved Memory]
    
    style A fill:#3498DB,stroke:#2980B9,color:#fff
    style C fill:#F1C40F,stroke:#F39C12,color:#000
    style D fill:#E67E22,stroke:#D35400,color:#fff
    style G fill:#F39C12,stroke:#E67E22,color:#fff
    style H fill:#27AE60,stroke:#229954,color:#fff
```

## 📚 Learning Objectives

1. ✅ **WriteToScratchPadTool**: Store information in agent memory
2. ✅ **ReadFromScratchPadTool**: Retrieve stored information
3. ✅ Building **stateful conversational agents**
4. ✅ Managing **conversation context and history**
5. ✅ **Persistent notes** across agent executions

---

## 🎯 What are ScratchpadTools?

**ScratchpadTools** provide memory capabilities for agents:
- 💾 **WriteToScratchPadTool**: Save information for later
- 📖 **ReadFromScratchPadTool**: Retrieve saved information
- 🔄 **Persistent Memory**: Information survives across agent calls
- 📝 **Context Tracking**: Maintain conversation state

**Use Cases**:
- 💬 Multi-turn conversations
- 📊 Progressive data collection
- 🎯 Personalized interactions
- 🔄 Workflow state management

---

## Step 1: Import Libraries

In [1]:
import sys
import json

sys.path.append('..')
from agent_helpers import (
    get_os_client,
    create_flow_agent,
    execute_agent,
    cleanup_resources
)

print("✅ Libraries imported!")

✅ Libraries imported!


## Step 2: Initialize Client

In [2]:
client = get_os_client()
print(f"✅ Connected to: {client.info()['cluster_name']}")

✅ Connected to: docker-cluster


## Step 3: Create Agent with Write Capability

In [3]:
# Agent with WriteToScratchPadTool
write_tools = [{
    "type": "WriteToScratchPadTool",
    "parameters": {
        "return_history": False
    }
}]

write_agent_id = create_flow_agent(
    client, "Memory_Writer_Agent",
    "Agent that writes information to scratchpad memory",
    write_tools
)
print(f"✅ Write agent created: {write_agent_id}")

   Registering flow agent: Memory_Writer_Agent...
   ✓ Agent registered: q7VZb5oBFJiTVjgy7JXH
✅ Write agent created: q7VZb5oBFJiTVjgy7JXH


## Step 4: Test Writing to Memory

In [4]:
# Write user preferences
parameters = {
    "notes": "User prefers: dashboard views, daily email reports, data in PST timezone"
}

print("✍️ Writing to scratchpad...")
response = execute_agent(client, write_agent_id, parameters)
print(json.dumps(response, indent=2))

✍️ Writing to scratchpad...
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Wrote to scratchpad: User prefers: dashboard views, daily email reports, data in PST timezone"
        }
      ]
    }
  ]
}


## Step 5: Write More Information

In [5]:
# Add conversation context
parameters = {
    "notes": "Previous query: Sales analysis for Q3 2025. User interested in regional breakdown.",
    "return_history": True  # Get all stored notes
}

print("✍️ Adding context...")
response = execute_agent(client, write_agent_id, parameters)
print(json.dumps(response, indent=2))

✍️ Adding context...
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad updated. Full content:\n- Previous query: Sales analysis for Q3 2025. User interested in regional breakdown."
        }
      ]
    }
  ]
}


## Step 6: Create Agent with Read Capability

In [6]:
# Agent with ReadFromScratchPadTool
read_tools = [{
    "type": "ReadFromScratchPadTool",
    "parameters": {
        "persistent_notes": ""  # Empty = read all notes
    }
}]

read_agent_id = create_flow_agent(
    client, "Memory_Reader_Agent",
    "Agent that reads information from scratchpad memory",
    read_tools
)
print(f"✅ Read agent created: {read_agent_id}")

   Registering flow agent: Memory_Reader_Agent...
   ✓ Agent registered: rLVab5oBFJiTVjgyKpUL
✅ Read agent created: rLVab5oBFJiTVjgyKpUL


## Step 7: Test Reading from Memory

In [7]:
# Read all stored information
parameters = {
    "persistent_notes": ""
}

print("📖 Reading from scratchpad...")
response = execute_agent(client, read_agent_id, parameters)
print(json.dumps(response, indent=2))

📖 Reading from scratchpad...
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad is empty."
        }
      ]
    }
  ]
}


## Step 8: Create Combined Read/Write Agent

In [8]:
# Agent with both read and write tools
memory_tools = [
    {
        "type": "ReadFromScratchPadTool",
        "parameters": {
            "persistent_notes": "Initial context: Customer service agent"
        }
    },
    {
        "type": "WriteToScratchPadTool",
        "parameters": {
            "return_history": True
        }
    }
]

memory_agent_id = create_flow_agent(
    client, "Full_Memory_Agent",
    "Agent with both read and write memory capabilities",
    memory_tools
)
print(f"✅ Full memory agent created: {memory_agent_id}")

   Registering flow agent: Full_Memory_Agent...
   ✓ Agent registered: rbVab5oBFJiTVjgyoJW4
✅ Full memory agent created: rbVab5oBFJiTVjgyoJW4


## Step 9: Simulate Conversation with Memory

In [9]:
# Conversation turn 1
print("💬 Turn 1: User provides order number")
parameters = {
    "notes": "Customer order #12345. Status: Shipped. Expected delivery: Nov 12."
}
response = execute_agent(client, memory_agent_id, parameters)
print(json.dumps(response, indent=2))

# Conversation turn 2
print("\n💬 Turn 2: User asks follow-up")
parameters = {
    "notes": "User requested tracking number. Provided: TRK-789-XYZ"
}
response = execute_agent(client, memory_agent_id, parameters)
print(json.dumps(response, indent=2))

# Read full history
print("\n📖 Reading conversation history:")
response = execute_agent(client, read_agent_id, {"persistent_notes": ""})
print(json.dumps(response, indent=2))

💬 Turn 1: User provides order number
{
  "inference_results": [
    {
      "output": [
        {
          "name": "WriteToScratchPadTool",
          "result": "Scratchpad updated. Full content:\n- Customer order #12345. Status: Shipped. Expected delivery: Nov 12."
        }
      ]
    }
  ]
}

💬 Turn 2: User asks follow-up
{
  "inference_results": [
    {
      "output": [
        {
          "name": "WriteToScratchPadTool",
          "result": "Scratchpad updated. Full content:\n- User requested tracking number. Provided: TRK-789-XYZ"
        }
      ]
    }
  ]
}

📖 Reading conversation history:
{
  "inference_results": [
    {
      "output": [
        {
          "name": "response",
          "result": "Scratchpad is empty."
        }
      ]
    }
  ]
}


## 🎓 Key Takeaways

### What We Learned:

1. **Scratchpad Tools**:
   - ✅ **WriteToScratchPadTool**: Store notes/context
   - ✅ **ReadFromScratchPadTool**: Retrieve stored information
   - ✅ **return_history**: Get all notes or just confirmation
   - ✅ **persistent_notes**: Initialize with context

2. **Memory Patterns**:
   ```python
   # Write pattern
   {
       "type": "WriteToScratchPadTool",
       "parameters": {
           "notes": "${parameters.notes}",
           "return_history": True
       }
   }
   
   # Read pattern
   {
       "type": "ReadFromScratchPadTool",
       "parameters": {
           "persistent_notes": "Initial context"
       }
   }
   ```

3. **Use Cases**:
   - 💬 **Multi-turn Conversations**: Remember past interactions
   - 📊 **Data Collection**: Gather info across multiple steps
   - 🎯 **Personalization**: Store user preferences
   - 🔄 **Workflow State**: Track progress in complex tasks

4. **Best Practices**:
   - ✅ **Structured Notes**: Use consistent format
   - ✅ **Clear Context**: Include timestamps, user IDs
   - ✅ **Scope Management**: Don't store excessive data
   - ✅ **Privacy**: Be careful with sensitive information

### Memory Management:

```python
# Good: Structured notes
notes = "Timestamp: 2025-11-09 | User: john@example.com | Action: Order inquiry | Order: #12345"

# Bad: Unstructured
notes = "user asked about order"
```

### Conversation Flow:

```
Turn 1: User asks question
  ↓
Write: Store question + context
  ↓
Turn 2: User asks follow-up
  ↓  
Read: Get previous context
  ↓
Write: Store new info + previous context
```

---

## 🧹 Cleanup

In [ ]:
# # cleanup_resources(
# #     client=client,
# #     agent_ids=[write_agent_id, read_agent_id, memory_agent_id]
# # )
# # print("✅ Cleanup complete!")

## 🚀 Next Steps

- **AgentTool**: Combine memory with multi-agent systems
- **RAGTool**: Add memory to RAG conversations
- **MLModelTool**: Build stateful conversational AI

📚 [ScratchpadTools Documentation](https://opensearch.org/docs/latest/ml-commons-plugin/agents-tools/tools/scratchpad-tools/)